<a href="https://colab.research.google.com/github/vvv94/indoor-localization/blob/main/localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing required Packages and import Python Modules

In [239]:
from time import sleep

Preparing the dataset

In [240]:
!rm -r sample_data dataset README.rtf Data.csv >/dev/null 2>&1
!mkdir -p dataset >/dev/null 2>&1
# WIFI Data
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=11El95X9yFnGqWhj7e9IulhYLN8SalJg8' -O ./dataset/AP_Data.csv -q
# Beacons Data
#!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1hDNQROcOcSDsbIUaVkTRdmYIrN81pNkA' -Ο ./dataset/Beacon_Data.csv -q
!mkdir -p results >/dev/null 2>&1

Load Dataset into a Table

In [290]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from functools import reduce

data = pd.read_csv("./dataset/AP_Data.csv", sep=';',index_col=0)

# Extract Location of AP's
AP_Locs = list(zip(data.iloc[0].values.astype(str).tolist(), data.iloc[1].values.astype(str).tolist()))
AP_Locs = AP_Locs[:len(AP_Locs)-2]
for idx,coords in enumerate(AP_Locs):
  x,y = coords
  x = x.replace(',', '.')
  y = y.replace(',', '.')
  AP_Locs[idx] = (x,y)
AP_Locs = np.asarray(AP_Locs,dtype=np.float64)


# Extract Location of AP's
Points_Locs = list(zip(data.iloc[2:,11].values.astype(str).tolist(),data.iloc[2:,12].values.astype(str).tolist()))
for idx,coords in enumerate(Points_Locs):
  x,y = coords
  x = x.replace(',', '.')
  y = y.replace(',', '.')
  Points_Locs[idx] = (x,y)
Points_Locs = np.asarray(Points_Locs,dtype=np.float64)
x, y = np.asarray(Points_Locs[:,0],dtype=str), np.asarray(Points_Locs[:,1],dtype=str)
Points_Locs = np.asarray([x[i]+'_'+y[i] for i in range(len(x))])

# Extract RSS of Points
RSS_Values = np.asarray(data.iloc[2:,0:11].values.astype(int).tolist(),np.int64)
RSS_Values = scale(RSS_Values)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(RSS_Values,Points_Locs,test_size=0.2,random_state=2020)
Y_train = Y_train.reshape(Y_train.shape[0],1)
Y_test = Y_test.reshape(Y_test.shape[0],1)

In [303]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold

#clf =DecisionTreeClassifier(criterion='entropy', max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None, max_leaf_nodes=None)
#fit = clf.fit(X_train,Y_train)
#y_pre =fit.predict(X_test)
#print(classification_report(Y_test,y_pre))

kf = KFold(n_splits=10,random_state=4, shuffle=False)

##Instantiatie my chosen model 
#clf = DecisionTreeClassifier(criterion='gini', max_depth=25, min_samples_split=2, min_samples_leaf=1, max_features=None, max_leaf_nodes=None)
clf = KNeighborsClassifier(5,weights='distance', p=2)

kf = KFold(n_splits=10,random_state=2020, shuffle=True)
for k, (train_index, test_index) in enumerate(kf.split(RSS_Values)):
 
    X_train, X_test = RSS_Values[train_index], RSS_Values[test_index]
    y_train, y_test = Points_Locs[train_index], Points_Locs[test_index]
    clf.fit(X_train, y_train)

    print("[fold {0}] score: {1:.5f}".format(k, clf.score(X_test, y_test)))
print("Fold,Mean ",k, clf.score(X_test, y_test).mean())


[fold 0] score: 0.00000
[fold 1] score: 0.00000
[fold 2] score: 0.00000
[fold 3] score: 0.00000
[fold 4] score: 0.00000
[fold 5] score: 0.00000
[fold 6] score: 0.00000
[fold 7] score: 0.00000
[fold 8] score: 0.00000
[fold 9] score: 0.00000
Fold,Mean  9 0.0


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Load Model

In [265]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers.core import Activation
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.optimizers import Adam

class Network:

  def __init__(self,input_size=11, num_classes=128):

      self.input_size = input_size
      self.output_size = num_classes
      self.dropout = 0.1

  def classifier(self):

      classifier = Sequential()
      classifier.add(Dense(units = 500, activation='relu', kernel_initializer='glorot_uniform', input_dim=self.input_size))
      classifier.add(Dropout(self.dropout))
      classifier.add(Dense(units = 100, activation='relu', kernel_initializer = 'glorot_uniform'))
      classifier.add(Dropout(self.dropout))
      classifier.add(Dense(units = 200, activation='relu', kernel_initializer = 'glorot_uniform'))
      classifier.add(Dropout(self.dropout))
      classifier.add(Dense(units = 50, activation='relu', kernel_initializer = 'glorot_uniform'))
      classifier.add(Dropout(self.dropout))
      classifier.add(Dense(units = self.output_size, kernel_initializer = 'uniform', activation = 'sigmoid'))
      classifier.compile(optimizer = Adam(), loss = 'categorical_crossentropy', metrics = ['accuracy'])
      model = classifier

      return model

In [261]:
# Load Model
model = Network(input_size=128, num_classes=128).classifier()

In [ ]:
# Train Model
model.fit(train_data, labels, epochs=20, batch_size=1)

In [ ]:
# Evaluate Model
loss, acc = c.evaluate(test_features, test_labels)
print loss, acc

More

In [ ]:
'''Project Parameters'''
DATASET_DIR = "./dataset/"
RESULTS_DIR = "./results"
DATASET_NAMES = [   
'P_SA',             # RSS values between the smartphone and 11 WiFi APs
'P_SA_Signatures',  # RSS values between the smartphone and 11 WiFi APs for calibration points (signatures)
'P_SA_Tests',       # RSS values between the smartphone and 11 WiFi APs for test points
'APLocs',           # x and y coordinates of 11 APs with RSS values stored in P_SA

'P_SB',             # RSS values between the smartphone and 11 WiFi APs
'P_SB_Signatures',  # RSS values between the smartphone and 46 Bluetooth beacons for calibration points (signatures)
'P_SB_Tests',       # RSS values between the smartphone and 46 Bluetooth beacons for test points
'BeaconLocs',       # x and y coordinates of 46 Bluetooth beacons with RSS values stored in P_SB

'MeasLocs',         # x and y coordinates of the smartphone with RSS values stored in P_SA and P_SB
'SignatureLocs',    # x and y coordinates of calibration points (signatures) with RSS values stored in P_SA_Signature and P_SB_Signature
'TestLocs',         # x and y coordinates of test points with RSS values stored in P_SA_Tests and P_SB_Tests
'P_Signatures',     # combination of RSS values for calibration points (signatures) - P_SA_Signature and P_SB_Signature
'P_Tests',          # combination of RSS values for test points - P_SA_Tests and P_SB_Tests
]

def pre_process(data_name):

  _colnames = []
  if data_name == "P_SA":
    _colnames=['WF1 RSS', 'WF2 RSS', 'WF3 RSS', 'WF4 RSS','WF5 RSS', 'WF6 RSS', 'WF7 RSS', 'WF8 RSS','WF9 RSS', 'WF10 RSS', 'WF11 RSS']
    data = pd.read_csv("./dataset/P_SA.csv", sep=';',names=_colnames,header=None)

  elif data_name == "P_SA_Signatures":
    _colnames=['WF1 RSS Sig.', 'WF2 RSS Sig.', 'WF3 RSS Sig.', 'WF4 RSS Sig.','WF5 RSS Sig.', 'WF6 RSS Sig.', 'WF7 RSS Sig.', 'WF8 RSS Sig.','WF9 RSS Sig.', 'WF10 RSS Sig.', 'WF11 RSS Sig.']
    data = pd.read_csv("./dataset/P_SA_Signatures.csv", sep=';',names=_colnames,header=None)

  elif data_name == "P_SA_Tests":
    _colnames=['WF1 RSS Test', 'WF2 RSS Test', 'WF3 RSS Test', 'WF4 RSS Test','WF5 RSS Test', 'WF6 RSS Test', 'WF7 RSS Test', 'WF8 RSS Test','WF9 RSS Test', 'WF10 RSS Test', 'WF11 RSS Test']
    data = pd.read_csv("./dataset/P_SA_Tests.csv", sep=';',names=_colnames,header=None)

  elif data_name == "APLocs":
    _colnames=['AP Coordinates', 'RSS Value in P_SA']
    data = pd.read_csv("./dataset/APLocs.csv", sep=';',names=_colnames,header=None)

  elif data_name == "P_SB":
    [_colnames.append('BT'+str(i)+' RSS') for i in range(1,47)]
    data = pd.read_csv("./dataset/P_SB.csv", sep=';',names=_colnames,header=None)

  elif data_name == "P_SB_Signatures":
    [_colnames.append('BT'+str(i)+' RSS Sig.') for i in range(1,47)]
    data = pd.read_csv("./dataset/P_SB_Signatures.csv", sep=';',names=_colnames,header=None)

  elif data_name == "P_SB_Tests":
    [_colnames.append('BT'+str(i)+' RSS Test') for i in range(1,47)]
    data = pd.read_csv("./dataset/P_SB_Tests.csv", sep=';',names=_colnames,header=None)

  elif data_name == "BeaconLocs":
    _colnames=['BT Coordinates', 'RSS Value in P_SB']
    data = pd.read_csv("./dataset/BeaconLocs.csv", sep='\t',names=_colnames,header=None)

  elif data_name == "MeasLocs":
    _colnames=['Smartphone Coordinates', 'RSS in P_SA and P_SB']
    data = pd.read_csv("./dataset/MeasLocs.csv", sep='	',names=_colnames,header=None)

  elif data_name == "SignatureLocs":
    _colnames=['Calibration Coordinates', 'RSS in P_SA and P_SB']
    data = pd.read_csv("./dataset/SignatureLocs.csv", sep=';',names=_colnames,header=None)

  elif data_name == "TestLocs":
    _colnames=['Test Coordinates', 'RSS in P_SA and P_SB']
    data = pd.read_csv("./dataset/TestLocs.csv", sep=';',names=_colnames,header=None)

  elif data_name == "P_Signatures":
    [_colnames.append('WF'+str(i)+' RSS Sign.') for i in range(1,12)]
    [_colnames.append('BT'+str(i)+' RSS Sign.') for i in range(12,58)]
    data = pd.read_csv("./dataset/P_Signatures.csv", sep=';',names=_colnames,header=None)

  elif data_name == "P_Tests":
    [_colnames.append('WF'+str(i)+' RSS Test') for i in range(1,12)]
    [_colnames.append('BT'+str(i)+' RSS Test') for i in range(12,58)]
    data = pd.read_csv("./dataset/P_Tests.csv", sep=';',names=_colnames,header=None)

  else:
    data = []

  return data

data = [pre_process(name) for idx, name in enumerate(DATASET_NAMES)]
dataset1 = [data[0], data[1], data[2], data[3]]
dataset2 = [data[4], data[5], data[6], data[7]]
dataset3 = [data[8], data[9], data[10], data[11], data[12]]